In [1]:
"""Set the problem (DTLZ# or WFG# or MaF#)"""
name = 'WFG5'

"""Set the number of objectives"""
M = 3

"""Set the number of generations"""
ngen = 633

In [2]:
"""imports"""
import numpy as np
import autograd.numpy as anp
import math
import maf
from pymoo.algorithms.moead_lws import MOEAD_LWS
from pymoo.optimize import minimize
from pymoo.factory import get_performance_indicator
import numpy as np
from pymoo.util.normalization import normalize as norm
import hvwfg
from pymoo.model.callback import Callback
from pymoo.factory import get_problem, get_crossover, get_mutation, get_reference_directions, get_selection
import os
if os.path.exists("MOEAD-LWS") is False:
    os.mkdir("MOEAD-LWS")

class MyCallback(Callback):
    def __init__(self) -> None:
        super().__init__()
        self.data["term_pop"] = None
    def notify(self, algorithm):
        self.data["term_pop"] = algorithm.pop

"""Set the 31 random seeds"""
seeds = np.linspace(1,1000000000,31)
seeds = [int(row) for row in seeds]

"""Set the RVs for each objective"""
def get_ref(M):
    if M==3:
        return get_reference_directions('das-dennis',3,n_partitions=13)
    elif M==5:
        return get_reference_directions('das-dennis',5,n_partitions=6)
    elif M==10:
        return get_reference_directions("multi-layer", 
                                        get_reference_directions("das-dennis", 10, n_partitions=3, scaling=1.0), 
                                        get_reference_directions("das-dennis", 10, n_partitions=2, scaling=0.5))
    elif M==15:
        return get_reference_directions("multi-layer", 
                                        get_reference_directions("das-dennis", 15, n_partitions=2, scaling=1.0), 
                                        get_reference_directions("das-dennis", 15, n_partitions=1, scaling=0.5))

if name[0]=='W' or name[0]=='w':
    problem = get_problem(name,n_obj=M,n_var=2*(M-1)+20)
elif name[0]=='D' or name[0]=='d':
    problem = get_problem(name,n_obj=M,n_var=(M-1)+20)
else:
    if name == 'MaF2':
        problem = maf.MaF2(n_obj=M,n_var=M-1+20)
    elif name == 'MaF3':
        problem = maf.MaF3(n_obj=M,n_var=M-1+20)
    elif name == 'MaF5':
        problem = maf.MaF5(n_obj=M,n_var=M-1+20)
    elif name == 'MaF7':
        problem = maf.MaF7(n_obj=M,n_var=M-1+20)
    elif name == 'MaF8':
        problem = maf.MaF8(n_obj=M)
    elif name == 'MaF9':
        problem = maf.MaF9(n_obj=M)
    elif name == 'MaF13':
        problem = maf.MaF13(n_obj=M,n_var=M-1+20)
    elif name == 'MaF14':
        problem = maf.MaF14(n_obj=M,n_var=20*M)
crossover = get_crossover('real_sbx',prob=0.9,eta=20)
if name in ['DTLZ1', 'DTLZ3']:
    mutation = get_mutation('real_pm',prob=1/problem.n_var,eta=5)
else:
    mutation = get_mutation('real_pm',prob=1/problem.n_var,eta=20)
ref_dirs = get_ref(M)
algorithm = MOEAD_LWS(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),mutation=mutation,eliminate_duplicates=True)
def optimize(seed_value):
    global problem,algorithm,ngen
    res = minimize(problem,algorithm,('n_gen', ngen),callback=MyCallback(),seed=seed_value)
    df = np.array(res.algorithm.callback.data["term_pop"])
    np.save('MOEAD-LWS/term_pop-'+name+'-'+str(M)+'-'+str(seed_value)+'.npy',df)

for i in range (31):
    print("Run:", i+1)
    optimize(seeds[i])
    
hv = []
if M == 3:
    p = 13
elif M == 5:
    p = 6
elif M == 10:
    p = 3
else:
    p = 2
if name in ['DTLZ2', 'DTLZ4', 'MaF1', 'MaF2', 'MaF13', 'MaF14']:
    lower = np.array([0.0]*M)
    upper = np.array([1.0]*M)
elif name in ['DTLZ1', 'DTLZ3', 'MaF3']:
    lower = np.array([0.0]*M)
    upper = np.array([10.0]*M)
elif name in ['MaF5']:
    lower = np.array([0.0]*M)
    upper = np.array([2**(M-i) for i in range(M)])
elif name in ['MaF7']:
    lower = np.array([0.0]*M)
    upper = np.array([1.0 for i in range(M)])
    upper[-1] = 2.0*M
elif name in ['MaF8','MaF9']:
    lower = np.array([0.0]*M)
    upper = np.array([2.0]*M)
elif name[0]=='W' or name[0]=='w':
    lower = np.array([0.0]*M)
    upper = np.array([2.0*(i+1) for i in range(M)])
for i in range(31):
    if os.path.exists('MOEAD-LWS/term_pop-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy'):
        temp = np.load('MOEAD-LWS/term_pop-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy', allow_pickle=True)
        temp = np.array([ind.F for ind in temp])
        temp = hvwfg.wfg(norm(temp, x_min=lower, x_max=upper), np.array([(p+1)/p]*M))
        hv.append(temp) 
print("Hypervolume for",name,":", np.median(hv))

Run: 1
Hypervolume for WFG5 : 0.5920931719766386
